## Train, Validation and Test Set

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?code_challenge=DZapn-dEA_F819OuQTkei5OGrw0gcfEqQPC5bvYqBHE&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


Enter verification code: 4/tQF7yIsW_AN8s_NGeeBZqFp5eg-pI1sFnVkjsFJXht1C8iPeD3bq8-w
If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [galli.giuly@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [4]:
!gcloud config set project reddit-master

Updated property [core/project].


In [0]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer


In [6]:
!gsutil cp gs://reddit_final_results/comments_posts_tokenized.csv .

Copying gs://reddit_final_results/comments_posts_tokenized.csv...
| [1 files][  1.7 GiB/  1.7 GiB]   73.2 MiB/s                                   
Operation completed over 1 objects/1.7 GiB.                                      


In [0]:
comments_posts_tokenized_df = pd.read_csv("comments_posts_tokenized.csv")

In [9]:
comments_posts_tokenized_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'subreddit', 'body', 'subreddit_id'], dtype='object')

In [10]:
categories = list(comments_posts_tokenized_df.columns.values)[2:]
print(categories)

['subreddit', 'body', 'subreddit_id']


In [0]:
X = comments_posts_tokenized_df['body']
y = comments_posts_tokenized_df['subreddit_id']

In [0]:
# Definint a fucntion that slit the dataset into three subsets: train, val and test
def train_dev_test_split(X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42, stratify=y_val)
    return dict(X_train=X_train, 
                X_val=X_val, 
                X_test=X_test, 
                y_train=y_train, 
                y_val=y_val, 
                y_test=y_test)

In [0]:
df_split = train_dev_test_split(X,y)

In [28]:
df_split.keys()

dict_keys(['X_train', 'X_val', 'X_test', 'y_train', 'y_val', 'y_test'])

## Vectorization

### Count vectorizer

In [0]:
count_vectors = CountVectorizer(ngram_range=(1,2), analyzer='word', strip_accents="unicode")

train_term_count = count_vectors.fit_transform(df_split['X_train'])
test_term_count = count_vectors.transform(df_split['X_val'])
test_term_count = count_vectors.transform(df_split['X_valid'])

train_term_count

In [0]:
v = train_term_count[0, :].toarray()

# compute the most common value
c = Counter(v[0])
print(c.most_common(1))

print(f"min: {v.min()}")
print(f"avg: {v.mean()}")
print(f"max: {v.max()}")
print(f"std: {v.std()}")
print(f"mode: {c.most_common(1)}")

### TFIDF

In [29]:
tfidf_vectors = TfidfVectorizer(ngram_range=(1,2),analyzer='word',strip_accents="unicode")

train_term_tfidf = tfidf_vectors.fit_transform(df_split['X_train'])
val_term_tfidf = tfidf_vectors.transform(df_split['X_val'])
test_term_tfidf = tfidf_vectors.transform(df_split['X_test'])

train_term_tfidf

KeyboardInterrupt: ignored

In [0]:
v = train_term_tfidf[0, :].toarray()

# compute the most common value
c = Counter(v[0])
print(c.most_common(1))

print(f"min: {v.min()}")
print(f"avg: {v.mean()}")
print(f"max: {v.max()}")
print(f"std: {v.std()}")
print(f"mode: {c.most_common(1)}")

In [0]:
feature_names = tfidf_vectors.get_feature_names()
